In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('../Data/IMDB Dataset.csv')


In [2]:
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [4]:
df.head(1)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive


In [5]:
#feature mapping of the sentiment 
feature_map = {'positive':1,'negative':0}
df['sentiment-map'] = df['sentiment'].map(feature_map)

In [6]:
df

,review,sentiment,sentiment-map
0,One of the other reviewers has mentioned that ...,positive,1
1,A wonderful little production. <br /><br />The...,positive,1
2,I thought this was a wonderful way to spend ti...,positive,1
3,Basically there's a family where a little boy ...,negative,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,1
...,...,...,...
49995,I thought this movie did a down right good job...,positive,1
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative,0
49997,I am a Catholic taught in parochial elementary...,negative,0
49998,I'm going to have to disagree with the previou...,negative,0


In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

In [8]:
df['review'][:10000]

0       One of the other reviewers has mentioned that ...
1       A wonderful little production. <br /><br />The...
2       I thought this was a wonderful way to spend ti...
3       Basically there's a family where a little boy ...
4       Petter Mattei's "Love in the Time of Money" is...
                              ...                        
9995    Fun, entertaining movie about WWII German spy ...
9996    Give me a break. How can anyone say that this ...
9997    This movie is a bad movie. But after watching ...
9998    This is a movie that was probably made to ente...
9999    Smashing film about film-making. Shows the int...
Name: review, Length: 10000, dtype: object

In [11]:
X = df['review'][:10000]
y = df['sentiment-map'][:10000]


In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
vectorizer = CountVectorizer(stop_words='english')

In [14]:
#using transform on test data to avoid data leakage
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test) 

In [15]:

model = Sequential()

In [16]:
X_train_vec.shape[1]

47529

In [17]:
model.add(Dense(64,activation='relu',input_shape=(X_train_vec.shape[1],)))
model.add(Dense(1,activation='sigmoid'))

In [18]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                3041920   
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 3041985 (11.60 MB)
Trainable params: 3041985 (11.60 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [20]:
X_train_dense = X_train_vec.toarray()

In [21]:
model.fit(X_train_dense, y_train, epochs=10, batch_size=32)

Epoch 1/10
250/250 [==============================] - 3s 10ms/step - loss: 0.3806 - accuracy: 0.8480
Epoch 2/10
250/250 [==============================] - 3s 10ms/step - loss: 0.0892 - accuracy: 0.9741
Epoch 3/10
250/250 [==============================] - 2s 9ms/step - loss: 0.0257 - accuracy: 0.9966
Epoch 4/10
250/250 [==============================] - 2s 9ms/step - loss: 0.0098 - accuracy: 0.9991
Epoch 5/10
250/250 [==============================] - 2s 9ms/step - loss: 0.0050 - accuracy: 0.9999
Epoch 6/10
250/250 [==============================] - 2s 9ms/step - loss: 0.0030 - accuracy: 1.0000
Epoch 7/10
250/250 [==============================] - 2s 9ms/step - loss: 0.0020 - accuracy: 1.0000
Epoch 8/10
250/250 [==============================] - 2s 9ms/step - loss: 0.0014 - accuracy: 1.0000
Epoch 9/10
250/250 [==============================] - 3s 10ms/step - loss: 0.0010 - accuracy: 1.0000
Epoch 10/10
250/250 [==============================] - 2s 9ms/step - loss: 7.6479e-04 - accuracy:

In [22]:
X_test_dense = X_test_vec.toarray()

In [23]:
loss, accuracy = model.evaluate(X_test_dense, y_test)
print("Test Accuracy:", accuracy)

63/63 [==============================] - 0s 5ms/step - loss: 0.5797 - accuracy: 0.8700
Test Accuracy: 0.8700000047683716
